In [0]:
!pip install textblob
!pip install spacy
!pip install lightgbm
!pip install xgboost
import nltk
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download('punkt')
!python -m spacy download en


    100% |████████████████████████████████| 13.9MB 2.0MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/s

In [0]:
import math
import pickle
import os
os.chdir("/content/gdrive/My Drive/Deep_Learning/")
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
import gc
from scipy import spatial
import spacy
import datetime as dt
from nltk.util import ngrams
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import lightgbm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import xgboost as xgb
from scipy.spatial.distance import cosine
from nltk.stem import PorterStemmer
spacy_nlp=spacy.load('en')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

FileNotFoundError: ignored

In [ ]:
##A function to preprocess the data
def pre_process(x,remove_stopwords=False):
    words = nltk.word_tokenize(x)
    words = [word.lower() for word in words if ((word.isalpha())|word.isnumeric())]
    words=list(map(lemmatizer.lemmatize, words))
    if remove_stopwords: 
        words=  [w for w in words if w not in stop_words]
        return words
    else:
        return words

In [0]:
#######Creating some functions for feature engineering###########

##Question mark feature
def question_mark(x):
    if "?" in x:
        return 1
    else:
        return 0
   
    
##Wh-words Feature
def wh_words(x):
    wh=['what','when','why','which','who','how','whose','whom','where']
    if len(list(set(x).intersection(wh)))!=0:
        return 1
    else:
        return 0

    
    
##Extracting root word of the sentence   
def root_word(x):
    lemma=[]
    temp=[sent.root for sent in en_nlp(x).sents]
    for i in temp:
        lemma.append(lemmatizer.lemmatize(str(i).lower()))
    return list(set(lemma))
   


##Creating N Grams
def create_ngrams(x,n):
    n_grams=[]
    for i in ngrams(x,n):
        n_grams.append(i)
    return list(set(n_grams))
    

    
##Creating pos tags
def pos_tags(x):
    pos=nltk.pos_tag(x)
    pos=list(set(pos))
    return pos


##POS tag of wh-word
def wh_pos(x):
    pos=nltk.pos_tag(x)
    pos=list(set(pos))
    for i in pos:
        if i[1] in['WDT','WP','WP$','WRB']:
            return str(i[1])
        return "None"           
    
    
    
##Has count words or not
def count_words(x):
    counting_words=['number','average','many','often','much','percent','percentage','ratio','distance',
                       'far','close','long']
    x=list(map(str,(x)))
    return len(set(x).intersection(counting_words))
    

##Get named entities
def ner(self,x):
    tagged=[]
    document=spacy_nlp(x)
    for element in document.ents:
        tagged.append((element,element.label_))
    return list(set(tagged))
    
        


In [0]:
#Initialize Global variables 
docIDFDict = {}
avgDocLength = 0

##Function to get passage corpus
def Get_Corpus(inputfile,corpusfile):
    f = open(inputfile,"r",encoding="utf-8")
    fw = open(corpusfile,"w",encoding="utf-8")
    for line in f:
        passage = line.strip().lower().split("\t")[2]
        fw.write(passage+"\n")
    f.close()
    fw.close()

    
    
##Function to generate IDF scores using the created corpus file    
def IDF_Generator(corpusfile, delimiter=' ', base=math.e) :

    global docIDFDict,avgDocLength

    docFrequencyDict = {}       
    numOfDocuments = 0   
    totalDocLength = 0

    for line in open(corpusfile,"r",encoding="utf-8") :
        doc = line.strip().split(delimiter)
        totalDocLength += len(doc)

        doc = list(set(doc)) # Take all unique words

        for word in doc : #Updates n(q_i) values for all the words(q_i)
            if word not in docFrequencyDict :
                docFrequencyDict[word] = 0
            docFrequencyDict[word] += 1

        numOfDocuments = numOfDocuments + 1
        if (numOfDocuments%5000==0):
            print(numOfDocuments)                

    for word in docFrequencyDict:  #Calculate IDF scores for each word(q_i)
        docIDFDict[word] = math.log((numOfDocuments - docFrequencyDict[word] +0.5) / (docFrequencyDict[word] + 0.5), base) #Why are you considering "numOfDocuments - docFrequencyDict[word]" instead of just "numOfDocuments"

    avgDocLength = totalDocLength / numOfDocuments

    print("NumOfDocuments : ", numOfDocuments)
    print("AvgDocLength : ", avgDocLength)


In [0]:

##Creating a similarity score based on BM25 formula
def GetBM25Score(x ,k1=1.5, b=0.75, delimiter=' ') :
    
    global docIDFDict,avgDocLength

    query_words= x['clean_query'].strip().lower().split(delimiter)
    passage_words = x['clean_passage'].strip().lower().split(delimiter)
    passageLen = len(passage_words)
    docTF = {}
    for word in set(query_words):   #Find Term Frequency of all query unique words
        docTF[word] = passage_words.count(word)
    commonWords = set(query_words) & set(passage_words)
    tmp_score = []
    for word in commonWords :   
        numer = (docTF[word] * (k1+1))   #Numerator part of BM25 Formula
        denom = ((docTF[word]) + k1*(1 - b + b*passageLen/avgDocLength)) #Denominator part of BM25 Formula 
        if(word in docIDFDict) :
            tmp_score.append(docIDFDict[word] * numer / denom)

    score = sum(tmp_score)
    return score

In [0]:
##Creating a match function 
def match(list1,list2):
    x=list(map(str, list1))
    y=list(map(str,list2))
    return len(set(x).intersection(y))
    

##Creating function for jaccardian similarity     
def get_jaccard_sim(list1, list2):
    a = set(list1) 
    b = set(list2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
    

In [0]:
glove_embeddings={}
emb_dim=100
def load_embeddings():
    temp={}
    global glove_embeddings,emb_dim
    file=open("glove.6B.100d.txt","r", encoding='utf-8',errors='ignore')
    for line in file:
        tokens= line.strip().split()
        word = tokens[0]
        vec = tokens[1:]
        vec = " ".join(vec)
        temp[word]=vec
    #Add Zerovec, this will be useful to pad zeros.
        temp["zerovec"] = "0.0 "*emb_dim
    file.close()
    for key in temp.keys():
        array=temp[key].split()
        array=list(map(float, array))
        glove_embeddings[key]=array
        

In [0]:
def get_embeddings(x):
    embeddings=[]
    for i in x:
        if i in glove_embeddings:
            embeddings.append(glove_embeddings[i])
        else:
            embeddings.append(glove_embeddings['zerovec'])
    return embeddings

In [0]:

data=pd.read_csv('data.tsv',delimiter='\t', header=None)
data.columns=['query_id','query','passage','label','passage_id']
subset=data.iloc[0:500000,:]
del data
gc.collect()

43

In [0]:

test=pd.read_csv('eval1_unlabelled.tsv',delimiter='\t', header=None)
test.columns=['query_id','query','passage','passage_id']
subset=test
del test
gc.collect()

30

In [0]:
##Question features
col1=features_1()
col2=features_2()
print("step 0")
GetCorpus('data.tsv','corpus.tsv')
IDF_Generator('corpus.tsv', delimiter=' ', base=math.e)
print("step1 ")
subset['ques_new']=subset['query'].apply(lambda x : col1.pre_process(x,remove_stopwords=True))
subset['ques_new_ws']=subset['query'].apply(lambda x : col1.pre_process(x,remove_stopwords=False))
print("step2 ")
subset['sent_new']=subset['passage'].apply(lambda x : col1.pre_process(x,remove_stopwords=True))
subset['sent_new_ws']=subset['passage'].apply(lambda x : col1.pre_process(x,remove_stopwords=False))
subset['ques_mark']=subset['query'].apply(lambda x: col1.question_mark(x))
print("step3 ")
subset['ques_length']=subset['ques_new'].apply(lambda x: len(x))
print("step3 ")
subset['wh_flag']=subset['ques_new_ws'].apply(lambda x: col1.wh_words(x))
print("step4 ")
#subset['ques_root_word']=subset['query'].apply(lambda x: col1.root_word(x))
print("step5 ")
subset['ques_unigram']=subset['ques_new'].apply(lambda x: col1.create_ngrams(x,1))
print("step6 ")
subset['ques_bigram']=subset['ques_new'].apply(lambda x: col1.create_ngrams(x,2))
print("step7 ")
subset['ques_trigram']=subset['ques_new'].apply(lambda x: col1.create_ngrams(x,3))
#print("step8 ")
#subset['ques_pos_tags']=subset['ques_new'].apply(lambda x: col1.pos_tags(x))
#print("step9 ")
#subset['ques_ner']=subset['query'].apply(lambda x : col.ner(x))
print("step10 ")
subset['ques_cnt_wrds']=subset['ques_new'].apply(lambda x: col1.count_words(x))
#print("step11 ")
#subset['wh_tag']=subset['ques_new_ws'].apply(lambda x: col1.wh_pos(x))
print("step12 ")
#subset['bm25']=subset.apply(lambda x: GetBM25Score(x), axis=1)


In [0]:
##Sentence features
print("step1 ")
subset['sent_length']=subset['sent_new'].apply(lambda x: len(x))
print("step2 ")
#subset['sent_root_word']=subset['passage'].apply(lambda x: col1.root_word(x))
print("step3 ")
subset['sent_unigram']=subset['sent_new'].apply(lambda x: col1.create_ngrams(x,1))
print("step4 ")
subset['sent_bigram']=subset['sent_new'].apply(lambda x: col1.create_ngrams(x,2))
print("step5 ")
subset['sent_trigram']=subset['sent_new'].apply(lambda x: col1.create_ngrams(x,3))
#print("step6 ")
#subset['sent_pos_tags']=subset['sent_new'].apply(lambda x: col1.pos_tags(x))
#print("step7 ")
#subset['sent_ner']=subset['passage'].apply(lambda x : col1.ner(x))
print("step8 ")
subset['numeric_flag']=subset.apply(lambda x: 1 if ((x['ques_cnt_wrds']>0)&(len([i for i in x['sent_new'] if i.isnumeric()])>0)) else 0, axis=1)



In [0]:
##Similarity features
print("step0 ")
load_embeddings()
#subset['rootword_match']=subset.apply(lambda x: col2.root_word_match(x),axis=1)
#subset['ner_match']=subset.apply(lambda x: col2.ner_match(x), axis=1)
print("step1 ")
subset['unigrams_match']=subset.apply(lambda x: col2.match(x['ques_unigram'],x['sent_unigram']), axis=1)
print("step2 ")
subset['bigrams_match']=subset.apply(lambda x: col2.match(x['ques_bigram'],x['sent_bigram']), axis=1)
print("step3 ")
subset['trigrams_match']=subset.apply(lambda x: col2.match(x['ques_trigram'],x['sent_trigram']), axis=1)
#subset['pos_tags_match']=subset.apply(lambda x: col2.pos_tags_match(x), axis=1)
#subset['ques_person']=subset.apply(lambda x: col2.ques_specific_ner(x,'Person'), axis=1)
#subset['ques_org']=subset.apply(lambda x: col2.ques_specific_ner(x,'Organisation'), axis=1)
#subset['ques_numerical']=subset.apply(lambda x: col2.ques_specific_ner(x,'Numerical'), axis=1)
#subset['ques_location']=subset.apply(lambda x: col2.ques_specific_ner(x,'Location'), axis=1)
#subset['ques_datetime']=subset.apply(lambda x: col2.ques_specific_ner(x,'Datetime'), axis=1)
#subset['sent_person']=subset.apply(lambda x: col2.sent_specific_ner(x,'Person'), axis=1)
#subset['sent_org']=subset.apply(lambda x: col2.sent_specific_ner(x,'Organisation'), axis=1)
#subset['sent_numerical']=subset.apply(lambda x: col2.sent_specific_ner(x,'Numerical'), axis=1)
#subset['sent_location']=subset.apply(lambda x: col2.sent_specific_ner(x,'Location'), axis=1)
#subset['sent_datetime']=subset.apply(lambda x: col2.sent_specific_ner(x,'Datetime'), axis=1)
subset['ratio_length']=subset['sent_length']/subset['ques_length']

print("step3 ")
subset['ques_embeddings']=subset['ques_new'].apply(lambda x: get_embeddings(x))
print("step4 ")
subset['sent_embeddings']=subset['sent_new'].apply(lambda x: get_embeddings(x))
print("step5 ")
subset['ques_embeddings']=subset['ques_embeddings'].apply(lambda x: np.array(x).mean(axis=0))
print("step6 ")
subset['sent_embeddings']=subset['sent_embeddings'].apply(lambda x: np.array(x).mean(axis=0))
print("step7 ")
subset['cos_similarity']=subset.apply(lambda x: cosine(x['ques_embeddings'],x['sent_embeddings']), axis=1)
print("step8 ")
subset['eu_similarity']=subset.apply(lambda x: np.sqrt(sum((x['ques_embeddings']-x['sent_embeddings'])**2)), axis=1)
print("step 9")
subset['jac_similarity']=subset.apply(lambda x: col2.get_jaccard_sim(x['ques_new_ws'],x['sent_new_ws']), axis=1 )


del subset['ques_embeddings']
del subset['sent_embeddings']


gc.collect()

In [0]:
subset.to_csv("test_adV2.tsv", sep="\t", index=False)
del subset
gc.collect()

0